In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
batchsize=8

In [3]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\Omkar\Desktop\Project\Projectdataset\Prepareddata\train',
                             target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training')

validation_data= train_datagen.flow_from_directory(r'C:\Users\Omkar\Desktop\Project\Projectdataset\Prepareddata\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Omkar\\Desktop\\Project\\Projectdataset\\Prepareddata\\train'

In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Omkar\Desktop\Project\Projectdataset\Prepareddata\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 79882 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 23s 0us/step


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint(r'C:\Users\Omkar\Desktop\Project\Projectdataset\model',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

C:\Users\Omkar\AppData\Local\Temp\ipykernel_11344\3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
7988/7988 [==============================] - ETA: 0s - loss: 0.2102 - accuracy: 0.9186
Epoch 1: val_loss improved from inf to 0.18332, saving model to C:\Users\Omkar\Desktop\Project\Projectdataset\model


INFO:tensorflow:Assets written to: C:\Users\Omkar\Desktop\Project\Projectdataset\model\assets


INFO:tensorflow:Assets written to: C:\Users\Omkar\Desktop\Project\Projectdataset\model\assets


7988/7988 [==============================] - 961s 120ms/step - loss: 0.2102 - accuracy: 0.9186 - val_loss: 0.1833 - val_accuracy: 0.9242 - lr: 0.0010
Epoch 2/5
7988/7988 [==============================] - ETA: 0s - loss: 0.1784 - accuracy: 0.9314
Epoch 2: val_loss improved from 0.18332 to 0.17649, saving model to C:\Users\Omkar\Desktop\Project\Projectdataset\model


INFO:tensorflow:Assets written to: C:\Users\Omkar\Desktop\Project\Projectdataset\model\assets


INFO:tensorflow:Assets written to: C:\Users\Omkar\Desktop\Project\Projectdataset\model\assets


7988/7988 [==============================] - 769s 96ms/step - loss: 0.1784 - accuracy: 0.9314 - val_loss: 0.1765 - val_accuracy: 0.9256 - lr: 0.0010
Epoch 3/5
7988/7988 [==============================] - ETA: 0s - loss: 0.1702 - accuracy: 0.9343

In [ ]:
# Model Evaluation

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)


In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)